# How to auto fine-tune a YOLOv11 model
---
Using a glove tracking model as an example, we show a simple way to use Optuna for fine-tuning our model.

## Pre-work

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`.

*** Most model training highly benefits from using a GPU-accelerated environment and this is no exception. We recommend using an L4 device, for example, in terms of cost/value ***


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Let's also make sure that we have enough RAM available.

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

*** Attention: *** At this point it is highly recommended that you mount your Google Drive and do not work in the simple Colab session because it is not permanent and you will lose any files you saved during the training session.

This is an example of mounting your Google Drive and working in a directory called "model-training" (which you had to previously created in your MyDrive).

If you want to mount your drive, uncomment the next cell and comment the following. If not, this demo will work on the ephemeral Colab session.

In [ ]:
"""
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
work_path = "/content/drive/MyDrive/model-training"
%cd {work_path}
"""

In [ ]:
work_path = "/content"

## Clone BaseballCV Repo, set as Current Directory and Install Requirements

In [ ]:
!git clone https://github.com/dylandru/BaseballCV.git


In [ ]:
%cd BaseballCV
!pip install -r requirements.txt


## Download Dataset from BaseballCV

This class utility will download the annotated dataset for glove, rubber, home and the ball.

In [ ]:
from scripts.load_tools import LoadTools

# Initialize LoadTools class
load_tools = LoadTools()

# Download images into unlabeled_ folder using alias
dataset_path = load_tools.load_dataset("baseball_rubber_home_glove")

print(dataset_path)

### Now we create the config file for the training classes.

In [ ]:
import yaml

data = {'train' :  f'{work_path}/BaseballCV/{dataset_path}/{dataset_path}/train/images',
        'val' :  f'{work_path}/BaseballCV/{dataset_path}/{dataset_path}/valid/images',
        'test' :  f'{work_path}/BaseballCV/{dataset_path}/{dataset_path}/test/images',
        'nc': 4,
        'names': ['glove','homeplate','baseball','rubber']
        }

# overwrite the data to the .yaml file
with open(f'{work_path}/BaseballCV/baseball_data.yaml', 'w') as f:
    yaml.dump(data, f)

# read the content in .yaml file
with open(f'{work_path}/BaseballCV/baseball_data.yaml', 'r') as f:
    hamster_yaml = yaml.safe_load(f)
    display(hamster_yaml)

Some cleaning of the label files.

In [ ]:
import os

# Directories for your label files
label_dirs = [
    f'{work_path}/BaseballCV/{dataset_path}/{dataset_path}/train/labels',
    f'{work_path}/BaseballCV/{dataset_path}/{dataset_path}/valid/labels',
    f'{work_path}/BaseballCV/{dataset_path}/{dataset_path}/test/labels'
]

# Function to remove lines with class 4
def filter_labels(label_dir):
    for label_file in os.listdir(label_dir):
        label_path = os.path.join(label_dir, label_file)
        with open(label_path, 'r') as file:
            lines = file.readlines()

        # Filter out annotations for class 4
        filtered_lines = [line for line in lines if not line.startswith('4')]

        # Rewrite the label file
        with open(label_path, 'w') as file:
            file.writelines(filtered_lines)

# Process all label directories
for label_dir in label_dirs:
    filter_labels(label_dir)

## Install Optuna
This will be the framework used for auto tuning the model.

In [ ]:
!pip install optuna

## Import the needed libraries.

Ultralytics & Optuna

In [ ]:
import optuna
from ultralytics import YOLO

## Define the Objective function
This function will execute the trials needed for finding the best parameters for the training of the model.

For each trial, there will be 5 epochs, and the function will return the params for the best accuracy by means of mAP50-95.

The function uses the YOLOv11 "yolo11x.pt" model for the pretrained model.

Trials will be run for different combinations of the following parameters:

lr: 1e-5 to 1e-2
batch size: 4 to 16
optimizer: AdamW, SGD

In [ ]:
# Define the objective function for Optuna
def objective(trial):
    # Define the hyperparameter search space
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [4, 8, 16])
    optimizer = trial.suggest_categorical('optimizer', ['AdamW', 'SGD'])

    # Load pretrained YOLO model
    model = YOLO("yolo11x.pt")
    print('Pretrained model loaded')

    # Train YOLO model with the trial's hyperparameters for 5 epochs
    model.train(data=f'{work_path}/BaseballCV/baseball_data.yaml',
                epochs=5,              # Only 5 epochs for the trial
                batch=batch_size,
                imgsz=640,
                optimizer=optimizer,
                lr0=lr,
                plots=True,
                val=True)

    # Evaluate the model and return validation accuracy
    metrics = model.val()
    return metrics.box.map  # Maximize accuracy (mean average precision)

## Running the trials

There will be executed 35 trials and the best results for accuracy will be stored in best_params.

If you want to modify the number if trials, modify the n_trials variable.

In [ ]:
# Run Optuna study with 35 trials
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=35)

# Get the best trial's hyperparameters
best_params = study.best_trial.params
print('Best hyperparameters found:', best_params)

In [ ]:
# Get the best trial's hyperparameters
best_params = study.best_trial.params
print('Best hyperparameters found:', best_params)

## Saving the parameters

For reference, a file with them are saved. Also, a training_instructions.txt is also saved.

In [ ]:
# Save the best hyperparameters to a file
with open('best_hyperparameters.yaml', 'w') as f:
    yaml.dump(best_params, f)

# Generate instructions for creating a model with these parameters
instructions = f"""
To create a YOLO model using the best hyperparameters found by Optuna, follow these steps:

1. Load the YOLO model:
   model = YOLO("yolo11x.pt")

2. Use the following parameters to train the model:

   - Learning rate (lr0): {best_params['lr']}
   - Batch size: {best_params['batch_size']}
   - Optimizer: {best_params['optimizer']}

3. Train the model with 25 epochs using this code:

   model.train(data='{work_path}/baseball_data.yaml',
               epochs=25,
               batch={best_params['batch_size']},
               imgsz=640,
               optimizer='{best_params['optimizer']}',
               lr0={best_params['lr']},
               plots=True,
               val=True)
"""

with open('training_instructions.txt', 'w') as f:
    f.write(instructions)

## Training the final model

Using the best parameters found in the previous trials, the model will be fully trained for 25 epochs.

*** (You can modify the numbers of epochs to your liking) ***

In [ ]:
# Load the YOLO model again
model = YOLO("yolo11x.pt")

# Train the final model using the best hyperparameters for 25 epochs
model.train(data=f'{work_path}/BaseballCV/baseball_data.yaml',
            epochs=25,                   # Train for 25 epochs with best params
            batch=best_params['batch_size'],
            imgsz=640,
            optimizer=best_params['optimizer'],
            lr0=best_params['lr'],
            plots=True,                   # Generate plots for final training
            val=True)

## Validation and showing the final model accuracy, (via mAP50-95)

In [ ]:
# Evaluate the final model
metrics = model.val()
print('Final model accuracy (mAP):', metrics.box.map)